In [2]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date

In [3]:
df_old = pd.read_csv("brd_covid_19.csv",parse_dates=[5])

In [4]:
df_old.sort_values(["bundesland","datum"]).head(15)

,bundesland,faelle,tod,lat,long,datum
34,Baden-Württemberg,14,0,48.5,9.0,2020-02-29
41,Baden-Württemberg,15,0,48.5,9.0,2020-03-01
51,Baden-Württemberg,19,0,48.5,9.0,2020-03-02
62,Baden-Württemberg,26,0,48.5,9.0,2020-03-03
76,Baden-Württemberg,44,0,48.5,9.0,2020-03-04
92,Baden-Württemberg,65,0,48.5,9.0,2020-03-05
108,Baden-Württemberg,91,0,48.5,9.0,2020-03-06
124,Baden-Württemberg,116,0,48.5,9.0,2020-03-07
140,Baden-Württemberg,182,0,48.5,9.0,2020-03-08
156,Baden-Württemberg,199,0,48.5,9.0,2020-03-09


In [101]:
df_old.to_csv ('brd_covid_19.csv', index = False, header=True)

In [124]:
today = date.today()

In [125]:
URL = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"

In [126]:
page = requests.get(URL)

In [127]:
soup = BeautifulSoup(page.content, 'html.parser')

In [128]:
results = soup.find_all("tr")

In [129]:
bundesland = []
faelle = []
tod = []
for idx,elm in enumerate(results):
    if idx > 0:
        bundesland.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[0])))
        faelle.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[0])
        if len(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()) > 1:
            tod.append(re.sub('<[^<]+?>', '', str(elm.find_all("td")[1])).split()[1].strip("(").strip(")"))
        else:
            tod.append("0")
    else:
        pass

In [130]:
lat=[48.5, 48.9, 53.4, 52.4, 53.1, 53.5, 50.7, 53.8, 53.87, 51.6,50, 49.4, 51.05, 52, 54.2, 50.85, 51.06]
long=[9, 11.4, 13.2, 13, 8.8, 10, 9, 12.5,9.5, 7.5,7.3, 6.97,13.5, 11.65,9.9, 11.13,10]


In [131]:
df=pd.DataFrame(data=[bundesland,faelle,tod,lat,long]).T
    

In [132]:
df["date"] = today

In [133]:
df.columns = ["bundesland","faelle","tod","lat","long","datum"]

In [134]:
df.tod = df.tod.replace(to_replace = "", value = "0", regex = True)

In [135]:
df.datum = pd.to_datetime(df.datum)

In [136]:
for idx,elm in enumerate(df.faelle):
    df.faelle[idx] = elm.replace(".","")

for idx,elm in enumerate(df.tod):
    df.tod[idx] = elm.replace(".","")

/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/karstenyan/opt/anaconda3/envs/nf/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [137]:
df.lat = df.lat.astype("float")
df.long = df.long.astype("float")
df.tod = df.tod.astype("int64")
df.faelle = df.faelle.astype("int64")

In [138]:
df=pd.concat([df_old,df]).reset_index().drop("index",axis=1)

In [139]:
df

,bundesland,faelle,tod,lat,long,datum
0,Baden-Württemberg,277,0,48.50,9.00,2020-03-12
1,Bayern,366,0,48.90,11.40,2020-03-12
2,Berlin,90,0,53.40,13.20,2020-03-12
3,Brandenburg,24,0,52.40,13.00,2020-03-12
4,Bremen,21,0,53.10,8.80,2020-03-12
5,Hamburg,48,0,53.50,10.00,2020-03-12
6,Hessen,48,0,50.70,9.00,2020-03-12
7,Mecklenburg-Vorpommern,17,0,53.80,12.50,2020-03-12
8,Niedersachsen,75,0,53.87,9.50,2020-03-12
9,Nordrhein-Westfalen,484,3,51.60,7.50,2020-03-12


In [140]:
df_old.to_csv ('old_data/brd_covid_19'+str(today)+".csv", index = False, header=True)

In [141]:
df.to_csv ('brd_covid_19.csv', index = False, header=True)